In [ ]:
import pandas as pd 
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("/kaggle/input/drug-classification/drug200.csv")

label_encoder = LabelEncoder()
df["Sex"] = label_encoder.fit_transform(df["Sex"])
df["BP"] = label_encoder.fit_transform(df["BP"])
df["Cholesterol"] = label_encoder.fit_transform(df["Cholesterol"])
df["Drug"] = label_encoder.fit_transform(df["Drug"])

x=df.drop(['Cholesterol','Drug'],axis=1).values
y=df[['Cholesterol','Drug']].values
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=101)

(age_train,sex_train,bp_train,na_to_k_train)=np.transpose(x_train)
(age_test,sex_test,bp_test,na_to_k_test)=np.transpose(x_test)
(cholestrol_train,drug_train)=(y_train).T
(cholestrol_test,drug_test)=(y_test).T

inputs_shape=(1,)
age=tf.keras.Input(shape=inputs_shape,name='age')
sex=tf.keras.Input(shape=inputs_shape,name='sex')
BP=tf.keras.Input(shape=inputs_shape,name='BP')
na_to_k=tf.keras.Input(shape=inputs_shape,name='na_to_k')

list_inputs=[age,sex,BP,na_to_k]


inputs=tf.keras.layers.concatenate(list_inputs)
x = tf.keras.layers.Dense(20, activation="linear")(inputs)
x = tf.keras.layers.Dropout(0)(x)
for i in range(0):
    x = tf.keras.layers.Dense(20, activation="linear")(x)
    x = tf.keras.layers.Dropout(0)(x)
   
outputs_1 = tf.keras.layers.Dense(1,activation='sigmoid')(x)
outputs_2=tf.keras.layers.Dense(5,activation='softmax')(x)
model = tf.keras.Model(inputs=list_inputs, outputs=[outputs_1,outputs_2], name="drug")

inputs_train={'age':age_train,'sex':sex_train,'BP':bp_train,'na_to_k':na_to_k_train}


batch_size=16
training_dataset = tf.data.Dataset.from_tensor_slices((inputs_train, cholestrol_train,drug_train))
training_dataset=training_dataset.batch(batch_size)

loss_obj_class = tf.keras.losses.SparseCategoricalCrossentropy()
loss_obj_cat = tf.keras.losses.BinaryCrossentropy()

error_cat = tf.keras.metrics.BinaryAccuracy()
error_class = tf.keras.metrics.SparseCategoricalAccuracy()



optimizer=tf.keras.optimizers.Adam(learning_rate=0.01)

loss_cat=[]
loss_class=[]

@tf.function
def train_step(inputs, y_cat, y_class):
    with tf.GradientTape() as tape:
        pred_cat, pred_class = model(inputs)
        cat_loss = loss_obj_cat(y_cat, pred_cat)
        class_loss = loss_obj_class(y_class, pred_class)

    gradients = tape.gradient([cat_loss, class_loss], model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    loss_cat.append(cat_loss)
    loss_class.append(class_loss)

    error_cat(y_cat, pred_cat)
    error_class(y_class, pred_class)
    
template = 'Epoch {:>3},' \
               ' cholestrol_class: {:>4.1%}, Drug_class: {:>5.1%}'

for epoch in range(500):
    
    for xx, yy, zz in training_dataset:
        train_step(xx, yy, zz)
        

    if (epoch + 1) % 10 == 0:

        print(template.format(epoch+1,
                            
                            error_cat.result(),
                            error_class.result()))

    error_cat.reset_states()
    error_class.reset_states()

    error_class.reset_states()
    error_cat.reset_states()
